 This file assesses the efficacy of Merrill Lynch Investment Clock by computing and comparing the annualized rate of return of stock, commodity, currency, and bond.

In [2]:
#import necessary packages
import pandas as pd
import os
import statistics
import math

In [3]:
#set working path
path="/Users/zhuoyangmeng/Desktop" #can change
os.chdir(path)

In [ ]:
#Helper functions

#function that converts a string list to a float list
def convert_to_float(str_list):
    for i in range(len(str_list)):
        str_list[i]=float(str_list[i])

#function that computes the average of a float list      
def average(lst):
    return sum(lst)/len(lst)

#function that gets the dates in an asset dataframe 
def get_date_list(data_frame):
    date_list=[]
    date_list=data_frame.iloc[:,0].tolist()
    for i in range(0,len(date_list)):
        date_list[i]=date_list[i].split(" ")[0]
    return date_list

#function that gets the prices in an asset dataframe
def get_price_list(data_frame):
    price_list=[]
    price_list=data_frame.iloc[:,1].tolist()
    for i in range(0,len(price_list)):
        price_list[i]=float(price_list[i])
    return price_list

#function that gets the indices of the start dates  
def find_start_index(date_list,str_list):
    index_list=[]
    starting_pos=0
    for k in range(0,len(date_list)):
        string=date_list[k]
        
        for i in range(starting_pos,len(str_list)):
            if string not in str_list[i]:
                continue
            else:
                index_list.append(i-1)
                starting_pos=i
                break
        
    return index_list

#function that gets the indices of the end dates
def find_end_index(date_list,str_list):
    index_list=[]
    end_pos=0
    
    for k in range(0,len(date_list)):
        string=date_list[k]
        
        for i in range(end_pos,len(str_list)):
            if i==len(str_list)-1:
                if string in str_list:
                    index_list.append(i)
                    end_pos=i
                break
           
            elif string in str_list[i] and string not in str_list[i+1]:
                index_list.append(i)
                end_pos=i
                break
            
            else:
                continue
    return index_list

#function that gets the prices on certain dates
def find_prices(price_list,index_list):
    new_price_list=[]
    for i in index_list:
        new_price_list.append(price_list[i])
    return new_price_list

#function that computes the annualized return rates
def compute_annualized_rate(start_price,end_price,durations):
    annualized_rate=[]
    for i in range(0,len(start_price)):
        rate=(end_price[i]-start_price[i])/(start_price[i]*durations[i])*12
        annualized_rate.append(rate)
    return annualized_rate

#function that computes the annualized return rates of currency 
def compute_money_rate(start_index,end_index,money_list,durations):
    annualized_rate=[]
    for i in range(0,len(start_index)):
        sum=0
        for k in range(start_index[i]+1,end_index[i]+1):
            sum+=money_list[k]
        annualized_rate.append(sum/(durations[i]*20)/100)
    return annualized_rate

#function that adjusts indices of a index list
def fix_index(index_list):
    new_index_list=[]
    for i in range(len(index_list)):
        new_index_list[i]=index_list[i]+1
    return new_index_list

#function that computes the annualized fluctuation
def compute_fluctuation(start_index,end_index,fluctuation_rate):
   fluctuation_list=[]
   for i in range(len(start_index)):
        sd=statistics.stdev(fluctuation_rate[start_index[i]:end_index[i]+1])
        fluctuation=sd*math.sqrt(240)
        fluctuation_list.append(fluctuation)
   return fluctuation_list

#function that computes the price change of stock over a certain period      
def compute_price_change(start_index,end_index,price_list):
    price_change_list=[]
    for i in range(len(start_index)):
        price_change=price_list[end_index[i]]-price_list[start_index[i]]
        price_change_list.append(price_change)
    return price_change_list

#function that determines type of market(bull,bear,shock)
def define_market(AR_list,price_change_list,fluctuation_list):
    market_list=[]
    for i in range(len(AR_list)):
        if (AR_list[i]>0 and abs(price_change_list[i])>abs(fluctuation_list[i])):
            market_list.append("Bull")
        elif (AR_list[i]<0 and abs(price_change_list[i])>abs(fluctuation_list[i])):
            market_list.append("Bear")
        else:
            market_list.append("Shock")
    return market_list

In [5]:
#store the start date, end date, duration and classification of each period 
start_date=["2002-12","2003-07","2003-09","2004-12","2006-01","2006-05","2008-03",
            "2008-10","2009-01","2009-08","2010-01","2010-07","2011-01","2011-12",
            "2013-07","2014-04","2014-07","2015-05","2016-02","2016-08","2017-02",
            "2018-05","2019-01","2019-07","2020-03","2020-08","2020-11","2021-07"]

end_date=["2003-06","2003-08","2004-11","2005-12","2006-04","2008-02","2008-09",
          "2008-12","2009-07","2009-12","2010-06","2010-12","2011-11","2013-06",
          "2014-03","2014-06","2015-04","2016-01","2016-07","2017-01","2018-04",
          "2018-12","2019-06","2020-02","2020-07","2020-10","2021-06","2021-09"]

durations=[7,2,15,13,4,22,7,3,7,5,6,6,11,19,9,3,10,9,6,6,15,8,6,8,5,3,8,3]
period=["recovery","boom","slowdown","recession","recovery","boom","slowdown",
        "recession","recovery","boom","slowdown","boom","slowdown","recovery",
        "slowdown","recovery","recession","recovery","recession","recovery","slowdown",
        "recession","recovery","recession","recovery","boom","slowdown","recession"]

In [ ]:
#read initial data of stock, commodity, currency, and bond
df_stock=pd.read_excel('stock.xlsx',dtype=str).iloc[:,[2,7]]

fluctuation_rate=(df_stock.iloc[:,[11]])["涨跌幅(%)"].tolist()

df_bond=pd.read_excel("bond.xlsx",dtype=str).iloc[:,[2,7]]

df_commodity=pd.read_excel("commodity.xlsx",dtype=str).iloc[:,[2,7]]

df_currency=pd.read_excel("currency.xlsx",dtype=str)

In [ ]:
#read dates for each asset
stock_dates=get_date_list(df_stock)
bond_dates=get_date_list(df_bond)
commodity_dates=get_date_list(df_commodity)
currency_dates=get_date_list(df_currency)

In [ ]:
#read prices for each asset
stock_prices=get_price_list(df_stock)
bond_prices=get_price_list(df_bond)
commodity_prices=get_price_list(df_commodity)
currency_prices=get_price_list(df_currency)

In [ ]:
#store indices of start dates and end dates into lists
start_index_stock=find_start_index(start_date,stock_dates)
end_index_stock=find_end_index(end_date,stock_dates)

start_index_bond=find_start_index(start_date,bond_dates)
end_index_bond=find_end_index(end_date,bond_dates)

start_index_commodity=find_start_index(start_date,commodity_dates)
end_index_commodity=find_end_index(end_date,commodity_dates)

start_index_currency=find_start_index(start_date,currency_dates)
end_index_currency=find_end_index(end_date,currency_dates)
start_index_currency.insert(0,15)

In [ ]:
#store openning prices and closing prices into lists
start_price_stock=find_prices(stock_prices,start_index_stock)
end_price_stock=find_prices(stock_prices,end_index_stock)

start_price_bond=find_prices(bond_prices,start_index_bond)
end_price_bond=find_prices(bond_prices,end_index_bond)

start_price_commodity=find_prices(commodity_prices,start_index_commodity)
end_price_commodity=find_prices(commodity_prices,end_index_commodity)


In [ ]:
#compute annualized return rates for all four assets
stock_rate=compute_annualized_rate(start_price_stock,end_price_stock,durations)
bond_rate=compute_annualized_rate(start_price_bond,end_price_bond,durations)
commodity_rate=compute_annualized_rate(start_price_commodity,end_price_commodity,durations)
currency_rate=compute_money_rate(start_index_currency,end_index_currency,currency_prices,durations)


In [ ]:
#write annualized return rates into excel
d={'stock':stock_rate,'bond':bond_rate,'commodity':commodity_rate,
'currency':currency_rate,'period':period}
df=pd.DataFrame(data=d)
df.to_excel("Annualized_Return.xlsx")

In [ ]:
#compute average annualized return rates for the assets during recovery
df_recovery=df[df["period"]=="recovery"]

stock_recovery=df_recovery["stock"].tolist()
convert_to_float(stock_recovery)
av_stock_recovery=average(stock_recovery)


bond_recovery=df_recovery["bond"].tolist()
convert_to_float(bond_recovery)
av_bond_recovery=average(bond_recovery)


commodity_recovery=df_recovery["commodity"].tolist()
convert_to_float(commodity_recovery)
av_commodity_recovery=average(commodity_recovery)


currency_recovery=df_recovery["currency"].tolist()
convert_to_float(currency_recovery)
av_currency_recovery=average(currency_recovery)

In [ ]:
#compute average annualized return rates for the assets during boom
df_boom=df[df["period"]=="boom"]

stock_boom=df_boom["stock"].tolist()
convert_to_float(stock_boom)
av_stock_boom=average(stock_boom)


bond_boom=df_boom["bond"].tolist()
convert_to_float(bond_boom)
av_bond_boom=average(bond_boom)


commodity_boom=df_boom["commodity"].tolist()
convert_to_float(commodity_boom)
av_commodity_boom=average(commodity_boom)


currency_boom=df_boom["currency"].tolist()
convert_to_float(currency_boom)
av_currency_boom=average(currency_boom)

In [ ]:
#compute average annualized return rates for the assets during slowdown
df_slowdown=df[df["period"]=="slowdown"]

stock_slowdown=df_slowdown["stock"].tolist()
convert_to_float(stock_slowdown)
av_stock_slowdown=average(stock_slowdown)


bond_slowdown=df_slowdown["bond"].tolist()
convert_to_float(bond_slowdown)
av_bond_slowdown=average(bond_slowdown)


commodity_slowdown=df_slowdown["commodity"].tolist()
convert_to_float(commodity_slowdown)
av_commodity_slowdown=average(commodity_slowdown)


currency_slowdown=df_slowdown["currency"].tolist()
convert_to_float(currency_slowdown)
av_currency_slowdown=average(currency_slowdown)

In [ ]:
#compute average annualized return rates for the assets during recession
df_recession=df[df["period"]=="recession"]

stock_recession=df_recession["stock"].tolist()
convert_to_float(stock_recession)
av_stock_recession=average(stock_recession)


bond_recession=df_recession["bond"].tolist()
convert_to_float(bond_recession)
av_bond_recession=average(bond_recession)


commodity_recession=df_recession["commodity"].tolist()
convert_to_float(commodity_recession)
av_commodity_recession=average(commodity_recession)


currency_recession=df_recession["currency"].tolist()
convert_to_float(currency_recession)
av_currency_recession=average(currency_recession)

In [ ]:
#write annualized return rates of the four assets into seperate excels
period_in_order=["recovery","recovery","recovery","recovery","recovery","recovery","recovery","recovery","recovery",
                 "boom","boom","boom","boom","boom",
                 "slowdown","slowdown","slowdown","slowdown","slowdown","slowdown","slowdown",
                 "recession","recession","recession","recession","recession","recession","recession"]

#stock
stock_ARlist=stock_recovery+stock_boom+stock_slowdown+stock_recession
d_s={"stock":stock_ARlist,"period":period_in_order}
df_stock_AR=pd.DataFrame(data=d_s)
df_stock_AR.to_excel("Stock_AR.xlsx")

#bond
bond_ARlist=bond_recovery+bond_boom+bond_slowdown+bond_recession
d_t={"bond":bond_ARlist,"period":period_in_order}
df_bond_AR=pd.DataFrame(data=d_t)
df_bond_AR.to_excel("Bond_AR.xlsx")

#commodity
commodity_ARlist=commodity_recovery+commodity_boom+commodity_slowdown+commodity_recession
d_c={"commodity":commodity_ARlist,"period":period_in_order}
df_commodity_AR=pd.DataFrame(data=d_c)
df_commodity_AR.to_excel("Commodity_AR.xlsx")

#currency
currency_ARlist=currency_recovery+currency_boom+currency_slowdown+currency_recession
d_m={"currency":currency_ARlist,"period":period_in_order}
df_currency_AR=pd.DataFrame(data=d_m)
df_currency_AR.to_excel("Currency_AR.xlsx")

In [ ]:
#determine type of markets(bull,bear,shock)
#compute annualized fluctuation rate
new_start_index_stock=fix_index(start_index_stock)
convert_to_float(fluctuation_rate)
fluctuation_list=compute_fluctuation(new_start_index_stock,end_index_stock,fluctuation_rate)

#compute price changes of stock during each period
price_change_list=compute_price_change(start_index_stock,end_index_stock,stock_prices)

#determine type of market
market_list=define_market(stock_rate,price_change_list,fluctuation_list)

#write market type information into excel
data_market={"market":market_list,"period":period}
df_market=pd.DataFrame(data=data_market)
df_market.to_excel("Market.xlsx")